Using some sota video classification model in transformers  
[VideoMAE](https://huggingface.co/docs/transformers/main/en/model_doc/videomae#transformers.VideoMAEForVideoClassification)

In [1]:
from transformers import AutoFeatureExtractor, AutoModelForVideoClassification

extractor = AutoFeatureExtractor.from_pretrained("meermoazzam41/videomae-base-finetuned-human-activity-classification")

model = AutoModelForVideoClassification.from_pretrained("meermoazzam41/videomae-base-finetuned-human-activity-classification")

/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/ansonlo/anaconda3/envs/torch/lib/python3.8/site-packages/transformers/models/videomae/feature_extraction_videomae.py:28: FutureWarning: The class VideoMAEFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use VideoMAEImageProcessor instead.
  warnings.warn(
